In [1]:
!pip install -U langchain langchain-openai langchain-community langchain-chroma chromadb pypdf


In [2]:
import langchain
print(langchain.__file__)

/usr/local/lib/python3.12/dist-packages/langchain/__init__.py


In [3]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory


In [4]:
import os
os.makedirs("/content/support_docs/UK_Workplace_Employment_Policy", exist_ok=True)


In [5]:
os.listdir("/content/support_docs")

['UK_Workplace_Employment_Policy.pdf',
 'UK_Workplace_Employment_Policy',
 'UK_Workplace_Employment_Polocy.pdf']

In [6]:
loader = DirectoryLoader(
    "/content/support_docs",
    glob="*.pdf",
    loader_cls=PyPDFLoader
)

docs = loader.load()
print(f"Loaded {len(docs)} pages")

Loaded 1 pages


In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

splits = splitter.split_documents(docs)
print(f"Created {len(splits)} chunks")


Created 3 chunks


In [8]:
import os
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"


In [9]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")


In [10]:
print(os.environ["OPENAI_API_KEY"][:10])


sk-proj-_h


In [11]:
embeddings = OpenAIEmbeddings()


In [12]:
from google.colab import userdata
import os

raw_key = userdata.get("OPENAI_API_KEY")

# IMPORTANT: strip whitespace and newlines
os.environ["OPENAI_API_KEY"] = raw_key.strip()


In [13]:
key = os.environ["OPENAI_API_KEY"]

print("Length:", len(key))
print("Contains newline:", "\n" in key or "\r" in key)
print("Starts with:", key[:10])


Length: 164
Contains newline: False
Starts with: sk-proj-_h


In [14]:

from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory="/content/chroma_db"
)

retriever = vectorstore.as_retriever()
print("Vector store ready")


Vector store ready


In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant answering questions based only on the provided document."),
    ("human", "Context:\n{context}\n\nQuestion:\n{question}")
])

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


In [20]:
def ask_question(question: str):
    docs = retriever.invoke(question)   # 👈 FIX IS HERE
    context = "\n\n".join(doc.page_content for doc in docs)

    response = llm.invoke(
        prompt.format_messages(
            context=context,
            question=question
        )
    )
    return response.content


In [21]:
print(
    ask_question(
        "What is the purpose of the UK workplace employment policy?"
    )
)


The purpose of the UK workplace employment policy is to outline the terms and conditions of employment to ensure a fair, safe, and legally compliant working environment in line with UK employment law.


In [ ]:
while True:
    question = input("\nYou: ")
    if question.lower() in ["exit", "quit"]:
        break

    answer = ask_question(question)
    print("\nBot:", answer)



You: no of holiday days?

Bot: Employees are entitled to 28 days paid annual leave per year, inclusive of UK bank holidays.

You: what about work from home?

Bot: Employees have the right to request flexible working from day one of employment. Working from home may be agreed on a full-time, hybrid, or occasional basis, depending on the suitability of the role and business needs. While working from home, employees must remain productive, be available during working hours, attend meetings, and maintain data security.
